# Getting started

The goal of `simpple` is to have a tiny [probablistic programming](https://en.wikipedia.org/wiki/Probabilistic_programming) language compatible with some of the most common sampling libraries.
That way, it is easy to write a model (including all priors) for `emcee`, and then run it with `nautilus`, for example.

In this tutorial, we will do a short demo by sampling a 3D normal distribution.


In [ ]:
import simpple
print(simpple.__version__)

## Model Definition

The two key components we need to specify in a Bayesian model are the prior distribution and the likelihood function.
Pretty much all sampling libraries require these two components, either separately for nested sampling, or combined in a posterior for MCMC.

In `simpple`, the prior is specified as a dictionary of `Distribution` objects.
Most common distributions are already implemented in scipy, so the recommended approach, inspired by [nautilus](https://nautilus-sampler.readthedocs.io/en/latest/guides/priors.html), is to simply wrap scipy distributions whenever possible.

```{note}
One specificity of scipy distributions compared to most other libarries is that the `uniform` distribution has parameters "lower" and "width" instead of "lower" and "upper".

For example, `uniform(-5, 10)` is a uniform distribution between -5 and 5.
```

In [ ]:
from scipy.stats import norm, uniform

from simpple.distributions import ScipyDistribution

parameters = {
    "x1": ScipyDistribution(uniform, -5, 10),
    "x2": ScipyDistribution(uniform, 0, 10),
    "x3": ScipyDistribution(norm(0, 10)),
}
print("Priors:")
print(parameters)

Next, we need a log-likelihood function that takes a dictionary of parameters and computes the likelihood.
In this simple 3D Gaussian example, we compare each parameter directly with a "data point" (`mean`), and the noise is correlated (defined by `cov`).

In [ ]:
from scipy.stats import multivariate_normal


def log_likelihood(params):
    """Log-likelihood function for a 3D normal distribution."""
    p = [params["x1"], params["x2"], params["x3"]]
    mean = [0.0, 3.0, 2.0]
    cov = [[1, 0.5, 0], [0.5, 1, 0], [0, 0, 1.0]]
    return multivariate_normal.logpdf(p, mean=mean, cov=cov)

It is now time to create our `simpple.Model` object.
The model needs to know what our priors and likelihood are.
It will then wrap them to provide:

- `log_prior(parameters)`: the prior distribution given a dictionary or an array of parameters
- `log_prob(parameters)`: the posterior distribution given a dictionary or an array of parameters
- `log_likelihood(parameters)`: a wrapper around our log-likelihood above to make it work with arrays and dictionaries
- `prior_transform(u)`: a prior transform from a unit hypercube to our parameter space

In [ ]:
from simpple.model import Model

model = Model(parameters, log_likelihood)

print(model)
test_point = [0, 1, 0]
print("Log-Prior", model.log_prior(test_point))
print("Log-Prior out of bounds", model.log_prior([-10, 3, 0]))
print("Log-likelihood", model.log_likelihood(test_point))
print("Log-posterior", model.log_prob(test_point))

## Prior Checks

A good thing to do before fitting any model is to check the prior.
To sample directly from the prior, we can either pass a uniform distribution through the `prior_transform` function or sample the `log_prior()` function of our model with `emcee`.

Here, we will test both approaches.
In practice, it's probably a good idea to test your prior transform if you plan on using nested sampling and your log-prior if you plan on using MCMC.

### Via prior transform

In [ ]:
import corner
import matplotlib.pyplot as plt
import numpy as np

rng = np.random.default_rng()
u = np.random.uniform(0, 1, size=(3, 100_000))
prior_samples = model.prior_transform(u)

fig = corner.corner(prior_samples.T, labels=model.keys())
fig.suptitle("Prior samples via prior transform")
plt.show()

### Via emcee

In [ ]:
import emcee

sampler = emcee.EnsembleSampler(
    nwalkers=100,
    ndim=3,
    log_prob_fn=model.log_prior,
)

In [ ]:
p0 = np.array(test_point) + 1e-4 * rng.standard_normal(size=(100, 3))
sampler.run_mcmc(p0, 1000, progress=True);

In [ ]:
chains = sampler.get_chain()

fig, axs = plt.subplots(3, 1)
for i in range(3):
    axs[i].plot(chains[:, :, i], "k-", alpha=0.1)
    axs[i].set_ylabel(model.keys()[i])
axs[-1].set_xlabel("Steps")
axs[0].set_title("Prior chains from emcee")
plt.show()

In [ ]:
flat_chains = sampler.get_chain(discard=200, flat=True)
fig = corner.corner(flat_chains, labels=model.keys())
fig.suptitle("Prior samples via emcee")
plt.show()

## Sampling with `emcee`

We can now sample the posterior with emcee. We can basically copy the previous subsection, but replace `log_prior` with `log_prob` in our sampler definition.

In [ ]:
sampler = emcee.EnsembleSampler(
    nwalkers=100,
    ndim=3,
    log_prob_fn=model.log_prob,
)

In [ ]:
p0 = np.array(test_point) + 1e-4 * rng.standard_normal(size=(100, 3))
sampler.run_mcmc(p0, 1000, progress=True);

In [ ]:
chains = sampler.get_chain()

fig, axs = plt.subplots(3, 1)
for i in range(3):
    axs[i].plot(chains[:, :, i], "k-", alpha=0.1)
    axs[i].set_ylabel(model.keys()[i])
axs[-1].set_xlabel("Steps")
axs[0].set_title("Prior chains from emcee")
plt.show()

In [ ]:
flat_chains = sampler.get_chain(discard=200, flat=True)
fig = corner.corner(flat_chains, labels=model.keys())
fig.suptitle("Prior samples via emcee")
plt.show()

## Sampling with `ultranest`

Next, let us try and sample the model with Ultranest.

In [ ]:
import ultranest

sampler = ultranest.ReactiveNestedSampler(
    model.keys(), model.log_likelihood, model.prior_transform
)

result = sampler.run(show_status=False)

In [ ]:
sampler.print_results()

In [ ]:
sampler.plot_corner()
plt.show()
sampler.plot_run()
plt.show()
sampler.plot_trace()
plt.show()

## Sampling with `nautilus`

The Nautilus sampler uses a slightly different format for the prior specification.
`simpple` can also interface with nautilus using `Model.nautilus_prior()`!

In [ ]:
from nautilus import Sampler

sampler = Sampler(model.nautilus_prior(), model.log_likelihood, n_live=1000)
sampler.run(verbose=True)

In [ ]:
points, log_w, log_l = sampler.posterior()
corner.corner(
    points,
    weights=np.exp(log_w),
    labels=model.keys(),
    plot_datapoints=False,
    range=np.repeat(0.999, len(model.parameters)),
)
plt.show()

## Comparison

Now that we have explored the posterior with several samplers, we can compare the resulting distributions.

In [ ]:
from matplotlib import patches

hist_kwargs = dict(density=True)

fig = corner.corner(
    points,
    weights=np.exp(log_w),
    labels=model.keys(),
    color="purple",
    hist_kwargs=hist_kwargs,
    plot_datapoints=False,
    range=np.repeat(0.999, len(model.parameters)),
)
data = np.array(result["weighted_samples"]["points"])
weights = np.array(result["weighted_samples"]["weights"])
corner.corner(
    data,
    weights=weights,
    color="red",
    hist_kwargs=hist_kwargs,
    plot_datapoints=False,
    range=np.repeat(0.999, len(model.parameters)),
    fig=fig,
)
corner.corner(
    flat_chains,
    weights=np.ones(flat_chains.shape[0]),
    color="k",
    hist_kwargs=hist_kwargs,
    plot_datapoints=False,
    fig=fig,
)


nautilus_patch = patches.Patch(color="purple", label="Nautilus")
ultranest_patch = patches.Patch(color="red", label="Ultranest")
emcee_patch = patches.Patch(color="k", label="Emcee")

fig.legend(
    handles=[nautilus_patch, ultranest_patch, emcee_patch],
    loc="upper right",  # You can also use 'upper left', 'lower right', etc.
    bbox_to_anchor=(0.98, 0.98),
)

plt.show()